<h1>Data Preprocessing<h1>

In [1]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from extract_dataframe import TweetDfExtractor
from extract_dataframe import read_json

_, tweet_list = read_json("../data/covid19.json")

In [2]:
tweets_df_extractor = TweetDfExtractor(tweet_list)
tweets_df = tweets_df_extractor.get_tweet_df()
tweets_df

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Fri Jun 18 17:55:49 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","🚨Africa is ""in the midst of a full-blown third...",0.166667,0.188889,en,548.0,612.0,ketuesriche,551,351,False,[],"[{'screen_name': 'TelGlobalHealth', 'name': 'T...",Mass
1,Fri Jun 18 17:55:59 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Dr Moeti is head of WHO in Africa, and one of ...",0.133333,0.455556,en,195.0,92.0,Grid1949,66,92,False,[],"[{'screen_name': 'globalhlthtwit', 'name': 'An...","Edinburgh, Scotland"
2,Fri Jun 18 17:56:07 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...",Thank you @research2note for creating this ama...,0.316667,0.483333,en,2.0,1.0,LeeTomlinson8,1195,1176,None,"[{'text': 'red4research', 'indices': [103, 116]}]","[{'screen_name': 'NHSRDForum', 'name': 'NHS R&...",None
3,Fri Jun 18 17:56:10 +0000 2021,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1580.0,899.0,RIPNY08,2666,2704,False,[],"[{'screen_name': 'HighWireTalk', 'name': 'The ...",None
4,Fri Jun 18 17:56:20 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",I think it’s important that we don’t sell COVA...,0.280000,0.620000,en,72.0,20.0,pash22,28250,30819,None,[],"[{'screen_name': 'PeterHotez', 'name': 'Prof P...",United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6527,Sat Jun 19 07:41:15 +0000 2021,"<a href=""http://twitter.com/download/android"" ...",RT @Givenkazeni: Zweli please just release the...,0.000000,0.400000,en,188.0,54.0,Mthatos_Vivi,447,1089,None,[],"[{'screen_name': 'Givenkazeni', 'name': 'le’Gi...",None
6528,Sat Jun 19 07:41:26 +0000 2021,"<a href=""http://twitter.com/download/android"" ...","Former Pfizer VP and Virologist, Dr. Michael Y...",0.086111,0.197222,en,1595.0,908.0,wayno_af007,2224,2739,False,[],"[{'screen_name': 'HighWireTalk', 'name': 'The ...","The boro, MA"
6529,Sat Jun 19 07:41:31 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","@Jenfeds73 @DcrInYYC Respectfully, veterinaria...",0.281250,0.506250,en,NaN,NaN,dublonothing,3000,4709,None,[],"[{'screen_name': 'Jenfeds73', 'name': 'Bubs 🇨🇦...","Los Angeles, CA"
6530,Sat Jun 19 07:41:45 +0000 2021,"<a href=""http://twitter.com/download/iphone"" r...","""Africa needs millions more doses here &amp; n...",0.125000,0.325000,en,143.0,122.0,DrAmirKhanGP,135163,1284,False,"[{'text': 'COVID19', 'indices': [120, 128]}]","[{'screen_name': 'WHOAFRO', 'name': 'WHO Afric...",Yorkshire and The Humber


In [3]:
tweets_df.isna().sum()

created_at               0
source                   0
original_text            0
polarity                 0
subjectivity             0
lang                     0
favorite_count        1747
retweet_count         1747
original_author          0
followers_count          0
friends_count            0
possibly_sensitive    2914
hashtags                 0
user_mentions            0
place                 2444
dtype: int64